In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
from mpl_toolkits.mplot3d import Axes3D
from pandas import Series

%matplotlib inline 


import plotly_express as px #plotly express is a library for drawing interactive figures
from glob import glob as lsdir
import os
import re
import datetime as dt

from uszipcode import SearchEngine #import this to pull zipcode data
search = SearchEngine(simple_zipcode=True) # set simple_zipcode=False to use rich info database

In [6]:
hotels = pd.read_csv('Datafiniti_Hotel_Reviews.csv') 
hotels.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,...,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,...,"2016-08-03T00:00:00Z,2016-07-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,NaN,NaN,Paula,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
1,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,...,"2016-08-02T00:00:00Z,2016-08-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,NaN,NaN,D,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
2,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,...,"2016-11-15T00:00:00Z,2016-08-23T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,NaN,NaN,Ron,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
3,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,...,"2016-05-21T00:00:00Z,2016-07-31T00:00:00Z",2.0,https://www.tripadvisor.com/Hotel_Review-g4118...,Currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,jaeem2016,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...
4,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,...,2016-07-31T00:00:00Z,5.0,https://www.tripadvisor.com/Hotel_Review-g4118...,I live in Md and the Aloft is my Home away fro...,ALWAYS GREAT STAY...,Laurel,MD,MamaNiaOne,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...


In [15]:
names = {'reviews.sourceURLS' : 'URL','reviews.rating' : 'rating', 'reviews.text' : 'text', 'reviews.title' : 'title'}
hotels.rename(names, inplace=True, axis=1)
hotels.columns


Index(['id', 'dateAdded', 'dateUpdated', 'address', 'categories',
       'primaryCategories', 'city', 'country', 'keys', 'latitude', 'longitude',
       'name', 'postalCode', 'province', 'reviews.date', 'reviews.dateSeen',
       'rating', 'reviews.sourceURLs', 'text', 'title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username', 'sourceURLs', 'websites'],
      dtype='object')

In [46]:
# create datafame of avg. review score grouped by hotel name
avg_hotels = hotels.groupby('name', as_index=False)['rating'].mean()

# add columns
avg_hotels['longitude'] = hotels.groupby('name', as_index=True)['longitude'].mean().values
avg_hotels['latitude'] = hotels.groupby('name', as_index=True)['latitude'].mean().values
avg_hotels['num_ratings'] = hotels.groupby('name', as_index=True)['rating'].count().values


# the code for these needs some help!
avg_hotels['zipcode'] = hotels.groupby('name')['postalCode']
avg_hotels['city'] = hotels.groupby('name').city


avg_hotels.head()

,name,rating,longitude,latitude,num_ratings,zipcode,city
0,1906 Lodge At Coronado Beach,5.000000,-117.176483,32.683517,1,"(1906 Lodge At Coronado Beach, [92118])","(1906 Lodge At Coronado Beach, [Coronado])"
1,250 Main Hotel,5.000000,-69.109400,44.101010,2,"(250 Main Hotel, [4841, 4841])","(250 Main Hotel, [Rockland, Rockland])"
2,AC Hotel Chicago Downtown,4.081818,-87.625630,41.893535,11,"(AC Hotel Chicago Downtown, [60611, 60611, 606...","(AC Hotel Chicago Downtown, [Chicago, Chicago,..."
3,AC Hotel Miami Beach,5.000000,-80.125114,25.805300,1,"(AC Hotel Miami Beach, [33140])","(AC Hotel Miami Beach, [Miami Beach])"
4,AC Hotel by Marriott Boston Downtown,5.000000,-71.061810,42.345090,1,"(AC Hotel by Marriott Boston Downtown, [2118])","(AC Hotel by Marriott Boston Downtown, [Boston])"


In [47]:
# map - trouble figuring it out (inspo from Jay's file in JAMRS in UVLT data stories)

mapbox_token = "pk.eyJ1Ijoianh6aG91MTkiLCJhIjoiY2p1a2xzYnB4MGl3ZjN5bXF1Z3FlZjdkMCJ9.MDXF150-61nXHGwAT52EWw"
px.set_mapbox_access_token(mapbox_token)

px.scatter_mapbox(avg_hotels, lat="latitude", lon="longitude", color="rating", size="num_ratings", 
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=40, zoom=10)

#color="categories", size="num_ratings", hover_name='name'

In [52]:
# create datafame of avg. review score grouped by state
state_avg = hotels.groupby('province', as_index=False)['rating'].mean()

# add columns
state_avg['longitude'] = hotels.groupby('province', as_index=True)['longitude'].median().values
state_avg['latitude'] = hotels.groupby('province', as_index=True)['latitude'].median().values
state_avg['num_ratings'] = hotels.groupby('province', as_index=True)['rating'].count().values




state_avg.head()


,province,rating,longitude,latitude,num_ratings
0,AK,3.333333,-149.93788,61.18119,21
1,AR,3.787879,-93.61175,35.28241,33
2,AZ,3.933962,-112.07390,33.59440,212
3,CA,4.024359,-119.72093,34.44910,1053
4,CO,4.074627,-104.90700,39.62643,134


In [ ]:
# might be good to map or plot the above?